In [87]:
from random import sample

import pandas as pd
import numpy as np
from dataclasses import dataclass


#define data classes:
@dataclass
class Run:
    id: int
    decay: float
    touchTransferFraction: float
    counts: list[int]
    occupancies: list[int]
    cdffs: list[int]
    anyCps: list[int]

@dataclass
class Sample:
        run: int
        startDay: int
        decay: float
        touchTransferFractions: float
        counts: list[int]
        occupancies: list[int]
        cdiffs: list[int]
        anyCps: list[int]
        


In [88]:
# read the simulated data into a dataframe.  Indexed on the run and sorted by tick.

data = pd.read_csv('data/sim_data.csv', index_col=['run']).sort_index().sort_values(by="tick")
data.head()

,count,tick,decayRate,surfaceTransferFraction,CDIFF,occupancy,anyCP
run,,,,,,,
16,23.0,90.0,0.590993,0.939448,0.0,15,2.0
8,0.0,90.0,0.789146,0.665336,0.0,15,5.0
6,2.0,90.0,0.728805,0.799395,0.0,16,3.0
19,5.0,90.0,0.781537,0.443818,2.0,12,2.0
2,0.0,90.0,0.539657,0.258963,0.0,20,4.0


In [89]:
#define a function to split the sequences into lists of n steps (we have 56 days of observed data)
#we will split the simulated data into sequences n long (90-136, 91-137, 92-138, etc.)
def split_sequence(seq, number_of_steps):
    print(seq)
    length = len(seq)
    last_start_position = length - number_of_steps
    sequences = []
    for i in range(last_start_position+1):
        sequence = seq[i:i+number_of_steps]
        sequences.append(sequence)
    return sequences
    

print(split_sequence(runs[3].counts, 56))

def split_sequences(run, number_of_steps):
    
    run_samples = []
    for i in np.arange(start, end):
        sample = Sample(run=run.id, startDay=i, decay=run.decay, touchTransferFractions=run.touchTransferFraction, 
            counts=split_sequence(run.counts), 
            occupancies=split_sequence(run.occupancy), 
            cdiffs=split_sequence(run.CDIFF), 
            anyCps=split_sequence(run.anyCp)
                )
        run_samples.append(sample)
    return run_samples

run
19    5.0
19    4.0
19    2.0
19    2.0
19    1.0
     ... 
19    0.0
19    0.0
19    1.0
19    1.0
19    1.0
Name: count, Length: 276, dtype: float64
[run
19    5.0
19    4.0
19    2.0
19    2.0
19    1.0
19    2.0
19    3.0
19    3.0
19    5.0
19    4.0
19    2.0
19    1.0
19    1.0
19    1.0
19    1.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    1.0
19    1.0
19    1.0
19    1.0
19    1.0
19    1.0
19    1.0
19    1.0
19    1.0
19    1.0
19    2.0
19    2.0
19    1.0
19    1.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
Name: count, dtype: float64, run
19    4.0
19    2.0
19    2.0
19    1.0
19    2.0
19    3.0
19    3.0
19    5.0
19    4.0
19    2.0
19    1.0
19    1.0
19    1.0
19    1.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0.0
19    0

In [84]:
# How many runs do we have and what are their lengths?
# for i in data.index.unique():
    #print(i, ": ", len(data.loc[i]))
    
runs = []

#create individual run objects for each run in the data.  Each run object will have a list of counts, occupancies, cdffs, and anyCps
for i in data.index.unique():
    #print(i)
    #print(data.loc[i].decayRate.iloc[1])
    run = Run(id=i, decay=data.loc[i].decayRate.iloc[0], touchTransferFraction=data.loc[i].surfaceTransferFraction.iloc[0], counts=data.loc[i]['count'], occupancies=data.loc[i].occupancy, cdffs=data.loc[i].CDIFF, anyCps=data.loc[i].anyCP)
    runs.append(run)


In [94]:
def mult(n,m):
    return n*m

test = mult

print(test(3,4))





# print(data.loc[3].count)
# print("------------")
# print(data.loc[3]['count'])

print(len(data.loc[3]['count']))

12
276
